In [161]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re

# to check if there are any null values
def is_null(df):
    print(df.isnull().any())
    return df[df.isnull().any(axis=1)].sort_values('user')

# to convert datetime rows in consistent datetime format 
def to_datetime_format(df,column_name):
    df[column_name] = pd.to_datetime(df[column_name])

# to replace any null values present in join date column  with the user's first transaction date
def replace_nan_to_1st_transaction(df,column_name):
    try:
        nan_rows = df[df[column_name].isnull()]
        first_transaction_date= nan_rows['transaction date'].groupby(nan_rows['user']).min()
        first_transaction_per_user = first_transaction_date.to_dict()
        df[column_name].loc[df[column_name].isnull()] = df.loc[df[column_name].isnull()].\
                                                        apply(lambda x: first_transaction_per_user[x[0]],axis=1)
    except:
        return ('no NaN values')

# to check minimum and maximum values for the given column
def check_min_max(df,column_name):
    print ("range of '{}' column = ({}, {})"\
           .format(column_name,df[column_name].min(),df[column_name].max()))

# date format check for datetime cloumns
def date_format_check(df,column_name):
    month_list = ['January','February','March','April','May','June','July','August','September'
                  ,'October','November','December']
    for i in range(len(df)):
        try:
            if re.match(r"("+'|'.join(month_list)+r")"+r"/(([0-2][0-9])|([3][0-1]))/([1-2]{1}[0-9]{3})",\
                        df[column_name].iloc[i]):
                continue
        except:
            print(i,df[column_name].iloc[i])
        

# region string check
def region_check(df):
    for i in range(len(df)):
        if (ord(df['region'].iloc[i]) >= ord('A')) and (ord(df['region'].iloc[i]) <= ord('Z')):
            continue
        else:
            print(i,df['region'].iloc[i])

# user id check
def user_check(df):
    for i in range(len(df)):
        if (df['user'].iloc[i] >0) and df['user'].iloc[i] == int(df['user'].iloc[i]):
            continue
        else:
            print(i,df['user'].iloc[i])

#sales amount check
def sales_amount_check(df):
    for i in range(len(df)):
        if df['sales amount'].iloc[i] == float(df['sales amount'].iloc[i]):
            continue
        else:
            print(i,df['user'].iloc[i])

# -------------------------------------------------------------------------------
# Check for "transaction_2013.csv" file

In [33]:
transaction_2013 = pd.read_csv('transactions_2013.csv',sep='\t')
df_2013 = pd.DataFrame(transaction_2013).sort_values('user')
df_2013.reset_index(drop=True,inplace=True)
df_2013.head()

,user,transaction date,sales amount,join date,region
0,1,May/11/2013,60.25,February/13/2012,A
1,1,February/13/2013,43.72,February/13/2012,A
2,2,October/14/2013,24.43,June/29/2004,B
3,2,February/07/2013,34.16,June/29/2004,B
4,2,September/16/2013,21.82,June/29/2004,B


In [34]:
is_null(df_2013)

user                False
transaction date    False
sales amount         True
join date            True
region              False
dtype: bool


,user,transaction date,sales amount,join date,region
1339,202,March/09/2013,NaN,January/25/2003,B
7648,3362,June/01/2013,23.47,NaN,B
7649,3362,December/08/2013,6.89,NaN,B
8732,4080,September/27/2013,56.52,NaN,D
14431,7253,July/06/2013,34.02,NaN,C
14432,7253,February/01/2013,59.50,NaN,C
14433,7253,June/11/2013,20.23,NaN,C
14434,7253,June/01/2013,11.31,NaN,C
14435,7253,December/13/2013,34.01,NaN,C


### As sales amount is "NaN", I assumed that that transaction got cancelled. So, there is no need to add that transaction for the given year.
### So, when someone calls the api for active user count, it should not reflect in my result.

In [35]:
df_2013.dropna(axis=0, subset=['sales amount'],inplace=True)
is_null(df_2013)

user                False
transaction date    False
sales amount        False
join date            True
region              False
dtype: bool


,user,transaction date,sales amount,join date,region
7648,3362,June/01/2013,23.47,NaN,B
7649,3362,December/08/2013,6.89,NaN,B
8732,4080,September/27/2013,56.52,NaN,D
14431,7253,July/06/2013,34.02,NaN,C
14432,7253,February/01/2013,59.50,NaN,C
14433,7253,June/11/2013,20.23,NaN,C
14434,7253,June/01/2013,11.31,NaN,C
14435,7253,December/13/2013,34.01,NaN,C


### As sales amount column should have value upto two decimal places, I will round sales amount upto two decimal.

In [36]:
df_2013['sales amount'] = df_2013['sales amount'].round(2)

In [37]:
# values that are zero after rounding up to two decimal places
df_2013[df_2013['sales amount'] <=0]

,user,transaction date,sales amount,join date,region
524,44,January/21/2013,0.00,January/01/2001,B
772,84,October/11/2013,-0.00,November/21/2008,B
1449,226,December/12/2013,-0.00,December/21/2006,C
1542,258,May/22/2013,-0.00,December/23/2005,B
2011,362,July/27/2013,0.00,March/10/2007,H
3007,706,July/21/2013,0.00,May/16/2012,D
3124,751,May/01/2013,0.00,October/28/2009,E
3222,770,October/14/2013,0.00,December/09/2010,L
3793,1073,February/18/2013,0.00,September/15/2009,M
4358,1417,April/28/2013,0.00,March/02/2007,B


### As the sales amount has zero values, according to me, those transactions are not valid.
### So, I will remove it from the dataframe

In [38]:
df_2013.drop(df_2013[df_2013['sales amount'] ==0].index,inplace=True)

### Check if all the user id is valid.

In [39]:
user_check(df_2013)

### Check if all "sales amount" values is valid

In [40]:
sales_amount_check(df_2013)

### Check for the minimum and maximum values of user and sales amount

In [41]:
check_min_max(df_2013,'user')
check_min_max(df_2013,'sales amount')

range of 'user' column = (1, 9999)
range of 'sales amount' column = (-13.12, 763.68)


### Check if there are any duplicate values.
### Actually, it will be wrong if I assume that the rows displayed below are really duplicated or not. As thinking from user's perspective, I can reorder as well. So, I cannot remove duplicate values from the dataframe.

In [89]:
 df_2013[df_2013.duplicated(keep=False)]

,user,transaction date,sales amount,join date,region
251,16,2013-06-21,4.94,2001-01-01,B
257,16,2013-06-21,4.94,2001-01-01,B
2035,369,2013-12-10,9.75,2005-11-19,B
2036,369,2013-12-10,9.75,2005-11-19,B
3195,768,2013-05-06,7.36,2015-02-16,L
3201,768,2013-05-06,7.36,2015-02-16,L
3207,768,2013-05-08,5.70,2015-02-16,L
3209,768,2013-05-08,6.53,2015-02-16,L
3211,768,2013-05-08,6.53,2015-02-16,L
3213,768,2013-05-08,5.70,2015-02-16,L


### Check if all the dates in "transaction date" is valid . If yes, convert in datetime format

In [42]:
date_format_check(df_2013,'transaction date')
to_datetime_format(df_2013,'transaction date')

### Check if all the dates in "join date" is valid . If yes, convert in datetime format 

### If any NaN values in "join date", replace it with the user's first transaction(purchased) date.

In [43]:
date_format_check(df_2013,'join date')

7623 nan
7624 nan
8706 nan
14380 nan
14381 nan
14382 nan
14383 nan
14384 nan


In [44]:
replace_nan_to_1st_transaction(df_2013,'join date')

### Change "join date" to date time format. 
### I did not converted it before because, transaction date had date in string format, so when choosing for first transaction, it sorted according to the alphabets occuring first in the string.

In [45]:
to_datetime_format(df_2013,'join date')

### Check if region value is valid

In [46]:
region_check(df_2013)

# Verify that there aren't any null values in the data frame

In [48]:
print(is_null(df_2013))
df_2013.head()

user                False
transaction date    False
sales amount        False
join date           False
region              False
dtype: bool
Empty DataFrame
Columns: [user, transaction date, sales amount, join date, region]
Index: []


,user,transaction date,sales amount,join date,region
0,1,2013-05-11,60.25,2012-02-13,A
1,1,2013-02-13,43.72,2012-02-13,A
2,2,2013-10-14,24.43,2004-06-29,B
3,2,2013-02-07,34.16,2004-06-29,B
4,2,2013-09-16,21.82,2004-06-29,B


# ALL THE CHECKS HAVE BEEN PERFORMED, SO NOW THE DATA CAN BE STORED IN THE DATABASE

## Perform similar steps for all other datasets

# ------------------------------------------------------------------------------------
# Check for "transaction_2014.csv" file


In [67]:
transaction_2014 = pd.read_csv('transactions_2014.csv',sep='\t')
df_2014 = pd.DataFrame(transaction_2014).sort_values('user')
df_2014.reset_index(drop=True,inplace=True)
df_2014.head()

,user,transaction date,sales amount,join date,region
0,1,August/19/2014,68.77,February/13/2012,A
1,2,September/11/2014,69.30,June/29/2004,B
2,2,April/09/2014,67.49,June/29/2004,B
3,2,May/29/2014,64.22,June/29/2004,B
4,2,January/21/2014,47.29,June/29/2004,B


In [68]:
df_2014['sales amount'] = df_2014['sales amount'].round(2)

In [69]:
df_2014[df_2014['sales amount'] ==0]

,user,transaction date,sales amount,join date,region
1410,217,December/11/2014,0.0,July/12/2003,B
1549,262,December/02/2014,0.0,March/26/2008,D
1766,311,May/15/2014,0.0,April/15/2011,G
2687,651,August/04/2014,0.0,June/13/2011,H
2768,675,May/29/2014,-0.0,April/15/2008,M
2910,747,November/28/2014,0.0,July/12/2001,O
3700,864,December/17/2014,0.0,December/13/2014,C
3769,864,December/16/2014,0.0,December/13/2014,C
4078,864,December/15/2014,0.0,December/13/2014,C
4202,864,December/21/2014,0.0,December/13/2014,C


In [70]:
df_2014.drop(df_2014[df_2014['sales amount'] ==0].index,inplace=True)

In [71]:
is_null(df_2014)

user                False
transaction date    False
sales amount         True
join date            True
region              False
dtype: bool


,user,transaction date,sales amount,join date,region
9259,3362,February/08/2014,23.43,NaN,B
9260,3362,September/21/2014,12.21,NaN,B
9632,3591,March/28/2014,NaN,August/20/2013,A
12493,4977,March/11/2014,NaN,December/20/2013,B


In [72]:
df_2014.dropna(axis=0, subset=['sales amount'],inplace=True)
is_null(df_2014)

user                False
transaction date    False
sales amount        False
join date            True
region              False
dtype: bool


,user,transaction date,sales amount,join date,region
9259,3362,February/08/2014,23.43,NaN,B
9260,3362,September/21/2014,12.21,NaN,B


In [73]:
user_check(df_2014)
sales_amount_check(df_2014)
check_min_max(df_2014,'user')
check_min_max(df_2014,'sales amount')

range of 'user' column = (1, 10000)
range of 'sales amount' column = (-339.19, 566.57)


In [74]:
date_format_check(df_2014,'transaction date')
to_datetime_format(df_2014,'transaction date')

In [75]:
date_format_check(df_2014,'join date')

9235 nan
9236 nan


In [76]:
replace_nan_to_1st_transaction(df_2014,'join date')
to_datetime_format(df_2014,'join date')

In [77]:
region_check(df_2014)


In [78]:
print(is_null(df_2014))
df_2014.head()

user                False
transaction date    False
sales amount        False
join date           False
region              False
dtype: bool
Empty DataFrame
Columns: [user, transaction date, sales amount, join date, region]
Index: []


,user,transaction date,sales amount,join date,region
0,1,2014-08-19,68.77,2012-02-13,A
1,2,2014-09-11,69.30,2004-06-29,B
2,2,2014-04-09,67.49,2004-06-29,B
3,2,2014-05-29,64.22,2004-06-29,B
4,2,2014-01-21,47.29,2004-06-29,B


# ----------------------------------------------------------------------------------
# Check for "transaction_2015.csv" file

In [80]:
transaction_2015 = pd.read_csv('transactions_2015.csv',sep='\t')
df_2015 = pd.DataFrame(transaction_2015).sort_values('user')
df_2015.reset_index(drop=True,inplace=True)
df_2015.head()

,user,transaction date,sales amount,join date,region
0,1,December/15/2015,160.60,February/13/2012,A
1,1,August/17/2015,50.09,February/13/2012,A
2,2,October/26/2015,6.06,June/29/2004,B
3,2,September/16/2015,37.20,June/29/2004,B
4,2,June/23/2015,35.88,June/29/2004,B


In [81]:
df_2015['sales amount'] = df_2015['sales amount'].round(2)

In [82]:
df_2015[df_2015['sales amount'] ==0]

,user,transaction date,sales amount,join date,region
93,10,February/16/2015,0.0,January/01/2001,C
127,11,June/26/2015,0.0,October/29/2008,A
130,11,August/21/2015,0.0,October/29/2008,A
884,124,April/22/2015,0.0,February/11/2010,J
1075,182,January/31/2015,0.0,April/03/2010,B
1291,267,March/23/2015,0.0,November/17/2008,B
1596,340,July/24/2015,0.0,April/18/2007,E
1903,488,September/07/2015,0.0,September/12/2010,N
2622,803,July/17/2015,0.0,June/26/2012,H
2627,806,June/10/2015,0.0,December/02/2010,A


In [83]:
df_2015.drop(df_2015[df_2015['sales amount'] ==0].index,inplace=True)

In [84]:
is_null(df_2015)

user                False
transaction date    False
sales amount        False
join date           False
region              False
dtype: bool


,user,transaction date,sales amount,join date,region


In [87]:
user_check(df_2015)
sales_amount_check(df_2015)

check_min_max(df_2015,'user')
check_min_max(df_2015,'sales amount')

range of 'user' column = (1, 9994)
range of 'sales amount' column = (0.01, 660.66)


In [90]:
date_format_check(df_2015,'transaction date')
to_datetime_format(df_2015,'transaction date')

In [91]:
date_format_check(df_2015,'join date')

In [92]:
replace_nan_to_1st_transaction(df_2015,'join date')
to_datetime_format(df_2015,'join date')

In [93]:
region_check(df_2015)

In [95]:
print(is_null(df_2015))
df_2015.head()

user                False
transaction date    False
sales amount        False
join date           False
region              False
dtype: bool
Empty DataFrame
Columns: [user, transaction date, sales amount, join date, region]
Index: []


,user,transaction date,sales amount,join date,region
0,1,2015-12-15,160.60,2012-02-13,A
1,1,2015-08-17,50.09,2012-02-13,A
2,2,2015-10-26,6.06,2004-06-29,B
3,2,2015-09-16,37.20,2004-06-29,B
4,2,2015-06-23,35.88,2004-06-29,B


# ---------------------------------------------------------------------------------------
# Check for "transaction_2016.csv" file

### This file had manny bad lines, so I used "usecols" parameter that only takes 5 columns, as all the extra "," were in the end of line, imported data will not have affected from it.


In [111]:
transaction_2016 = pd.read_csv('transactions_2016.csv',sep=',',usecols=[0,1,2,3,4])
df_2016 = pd.DataFrame(transaction_2016).sort_values('user')
df_2016.rename(columns={'transaction_date':'transaction date',
                'sales_amount':'sales amount', 'join_date': 'join date'}, inplace =True)
df_2016.reset_index(drop=True,inplace=True)
df_2016.head()

,user,transaction date,sales amount,join date,region
0,2,18-03-2016,6.96,29-06-2004,B
1,2,15-01-2016,27.04,29-06-2004,B
2,2,10-02-2016,58.83,29-06-2004,B
3,2,15-02-2016,39.48,29-06-2004,B
4,2,19-04-2016,54.80,29-06-2004,B


In [112]:
is_null(df_2016)

user                False
transaction date    False
sales amount        False
join date            True
region              False
dtype: bool


,user,transaction date,sales amount,join date,region
2319,4080,26-02-2016,59.36,NaN,D


In [113]:
df_2016['sales amount'] = df_2016['sales amount'].round(2)
df_2016[df_2016['sales amount'] ==0]

,user,transaction date,sales amount,join date,region
480,361,02-04-2016,0.0,18-07-2010,B
958,1161,16-04-2016,0.0,03-01-2015,J
959,1161,30-04-2016,0.0,03-01-2015,J
960,1161,12-03-2016,0.0,03-01-2015,J
961,1161,02-04-2016,0.0,03-01-2015,J
1251,1746,28-01-2016,0.0,21-02-2014,A
1359,1972,22-03-2016,0.0,01-07-2013,A
1405,2067,06-04-2016,0.0,02-09-2013,A
1707,2823,10-01-2016,0.0,17-05-2009,B
2369,4218,14-03-2016,0.0,05-02-2015,C


In [114]:
df_2016.drop(df_2016[df_2016['sales amount'] ==0].index,inplace=True)

In [115]:
user_check(df_2016)
sales_amount_check(df_2016)

check_min_max(df_2016,'user')
check_min_max(df_2016,'sales amount')

range of 'user' column = (2, 9984)
range of 'sales amount' column = (0.01, 674.18)


In [116]:
date_format_check(df_2016,'transaction date')
to_datetime_format(df_2016,'transaction date')

In [130]:
date_format_check(df_2016,'join date')

2305 nan


In [117]:
replace_nan_to_1st_transaction(df_2016,'join date')
to_datetime_format(df_2016,'join date')

In [118]:
region_check(df_2016)

In [119]:
print(is_null(df_2016))
df_2016.head()

user                False
transaction date    False
sales amount        False
join date           False
region              False
dtype: bool
Empty DataFrame
Columns: [user, transaction date, sales amount, join date, region]
Index: []


,user,transaction date,sales amount,join date,region
0,2,2016-03-18,6.96,2004-06-29,B
1,2,2016-01-15,27.04,2004-06-29,B
2,2,2016-10-02,58.83,2004-06-29,B
3,2,2016-02-15,39.48,2004-06-29,B
4,2,2016-04-19,54.80,2004-06-29,B


# -----------------------------------------------------------------------------

# Concatenate all the dataframes into a single dataframe which will then be used to populate the MySQL database hosted on AWS EC2 instance running on  the lamp server

In [120]:
frames = [df_2013,df_2014,df_2015,df_2016]
df = pd.concat(frames)

In [121]:
df.sort_values('user',inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,user,transaction date,sales amount,join date,region
0,1,2013-05-11,60.25,2012-02-13,A
1,1,2014-08-19,68.77,2012-02-13,A
2,1,2015-08-17,50.09,2012-02-13,A
3,1,2015-12-15,160.60,2012-02-13,A
4,1,2013-02-13,43.72,2012-02-13,A


In [122]:
len(df)

51263

# POPULATE MySQL database using pymysql library

In [147]:
import pymysql
def connect_sql():
    server = pymysql.connect(
        host='HOSTNAME',
        user='USERNAME',
        password='PASSWORD',
        database='DATABASE_NAME')
    return server,server.cursor()

In [158]:
server,cursor = connect_sql()
sql = "CREATE DATABASE IF NOT EXISTS 26capital;"
cursor.execute(sql)
sql = "USE 26capital;"
cursor.execute(sql)
try:
    sql = '''CREATE TABLE transaction_data(id int NOT NULL AUTO_INCREMENT,
    user int NOT NULL ,
    transaction_date DATE NOT NULL,
    sales_amount  FLOAT,
    join_date DATE NOT NULL,
    region VARCHAR(4) NOT NULL,PRIMARY KEY(id,user));'''
    cursor.execute(sql)
    server.commit()
except pymysql.DatabaseError as error:
    print('error:',error)
sql = 'show tables;'
cursor.execute(sql)
print('table name:',cursor.fetchall())
cursor.close()

table name: (('transaction_data',),)


In [160]:
server,cursor = connect_sql()
for i in range(len(df)):
    try:
        per_row = df.iloc[i].to_dict()
        sql = ("INSERT INTO transaction_data(user,transaction_date,sales_amount,join_date,region) \
               VALUES('{}','{}','{}','{}','{}');"\
               .format(per_row['user'],per_row['transaction date'],\
                       per_row['sales amount'],per_row['join date'],per_row['region'])
              )
        cursor.execute(sql)
        server.commit()
    except pymysql.DatabaseError as error:
        print('error:',error)
cursor.close()